# Raw Material Prices <br>
The script handles data extraction, processing, and preparation for three distinct Excel files required for various system uploads, including API integration, manual database updates, and bulk uploads. The data flow includes reading raw data from Excel sheets, combining it with supplier price requests, processing it, and outputting the required formatted Excel files.

In [1]:
import pandas as pd
import os
from datetime import datetime
import re
from openpyxl import load_workbook

In [2]:
# Paths
base_file_path = r'C:\Users\plmala\OneDrive - Coloplast A S\Desktop\Python\PPM\Price Updates (Python)\0_Price Updates\Single Items.xlsm'
output_folder_path = r'C:\Users\plmala\OneDrive - Coloplast A S\Desktop\Python\PPM\Price Updates (Python)\0_Price Updates\New Files'
manual_update_folder_path = r'C:\Users\plmala\OneDrive - Coloplast A S\Desktop\Python\PPM\Price Updates (Python)\0_Price Updates\Manual Update'
api_template_folder_path = r'C:\Users\plmala\OneDrive - Coloplast A S\Desktop\Python\PPM\Price Updates (Python)\0_Price Updates\API Template'
massive_upload_folder_path = r'C:\Users\plmala\OneDrive - Coloplast A S\Desktop\Python\PPM\Price Updates (Python)\0_Price Updates\Massive Upload'

In [ ]:
# Check, load, define and read
if not os.path.exists(base_file_path):
    raise FileNotFoundError(f"Base file not found: {base_file_path}")
wb = load_workbook(base_file_path, keep_vba=True, keep_links=False)

def read_sheet_to_df(workbook, sheet_name):
    ws = workbook[sheet_name]
    data = ws.values
    columns = next(data)
    return pd.DataFrame(data, columns=columns)

base_file_df = read_sheet_to_df(wb, 'Template')
hubble_data_df = read_sheet_to_df(wb, 'Hubble data')
item_branch_df = read_sheet_to_df(wb, 'Item Branch')

c:\Users\plmala\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
# Clean the data
base_file_df.columns = base_file_df.columns.str.strip()
hubble_data_df.columns = hubble_data_df.columns.str.strip()
item_branch_df.columns = item_branch_df.columns.str.strip()
columns_to_convert = [
    'Item', '2ND ITEM NUMBER', 'SUPPLIER NUMBER', 'ITEM NUMBER', 'SUPPLIER DESCRIPTION',
    'BRANCH PLANT', 'Supplier Number', 'Item Number', 'Branch Plant'
]

for df in [base_file_df, hubble_data_df, item_branch_df]:
    for col in columns_to_convert:
        if col in df.columns:
            df[col] = df[col].astype(str).str.upper().str.strip()
base_file_df['Item'] = base_file_df['Item'].replace(['NONE', '', 'N/A', 'UNKNOWN', 'NAN'], pd.NA)
base_file_df = base_file_df.dropna(subset=['Item'])
base_file_df['New Price'] = base_file_df['New Price'].astype(str).str.replace(',', '.')
base_file_df['New Price'] = pd.to_numeric(base_file_df['New Price'], errors='coerce').round(4)

In [5]:
# Fill mandatory columns with default values if empty
base_file_df['Main Reason 1'] = base_file_df['Main Reason 1'].fillna('Index Driven')
base_file_df['Weight (%) Reason 1'] = base_file_df['Weight (%) Reason 1'].fillna(100)
base_file_df['Main Reason 2'] = base_file_df['Main Reason 2'].fillna('')
base_file_df['Weight (%) Reason 2'] = base_file_df['Weight (%) Reason 2'].fillna('')
base_file_df['Main Reason 3'] = base_file_df['Main Reason 3'].fillna('')
base_file_df['Weight (%) Reason 3'] = base_file_df['Weight (%) Reason 3'].fillna('')

In [ ]:
# Prepare 'Effective date' column
if 'Effective date' in base_file_df.columns:
    base_file_df['Effective date'] = pd.to_datetime(base_file_df['Effective date'], errors='coerce')
# Convert 'AMOUNT - UNIT COST' to float and 'DATE - EFFECTIVE' to datetime
def extract_numbers(x):
    nums = re.findall(r"[-+]?\d*\.\d+|\d+", str(x).replace(',', '.'))
    return round(float(nums[0]), 4) if nums else 0.0000

hubble_data_df['AMOUNT - UNIT COST'] = hubble_data_df['AMOUNT - UNIT COST'].apply(extract_numbers)
hubble_data_df['DATE - EFFECTIVE'] = pd.to_datetime(hubble_data_df['DATE - EFFECTIVE'], errors='coerce')

In [7]:
# Merge, map and debug
merged_df = pd.merge(base_file_df, hubble_data_df, left_on='Item', right_on='2ND ITEM NUMBER', how='left')
unmatched_items = merged_df[merged_df['2ND ITEM NUMBER'].isna()]['Item'].unique()
if unmatched_items.size > 0:
    print("Items not found in hubble_data_df:")
    print(unmatched_items)
merged_df['AMOUNT - UNIT COST'] = merged_df['AMOUNT - UNIT COST'].fillna(0.0)
merged_df['DATE - EFFECTIVE'] = merged_df['DATE - EFFECTIVE'].fillna(pd.NaT)
item_branch_df = item_branch_df[['SUPPLIER NUMBER', 'ITEM NUMBER', 'BRANCH PLANT']]
item_branch_df.columns = ['Supplier Number', 'Item Number', 'Branch Plant']
item_branch_df = item_branch_df.apply(lambda x: x.str.strip())
merged_df['Supplier Number'] = merged_df['SUPPLIER NUMBER'].str.strip()
merged_df['Item Number'] = merged_df['Item'].str.strip()
merged_df['Branch Plant'] = merged_df['BRANCH PLANT'].str.strip()
key_cols = ['Supplier Number', 'Item Number', 'Branch Plant']
merged_df = pd.merge(
    merged_df,
    item_branch_df.drop_duplicates(subset=key_cols),
    on=key_cols,
    how='left',
    indicator=True
)
merged_df['620'] = ''

Items not found in hubble_data_df:
['16001601']


In [ ]:
# Define prefixes
branch_prefixes = ['790', '620', '110', '490', '351', '711']
def assign_620(df):
    grouped = df.groupby(['Supplier Number', 'Item Number'])
    for (supplier_number, item_number), group in grouped:
        for prefix in branch_prefixes:
            matches = group[
                group['Branch Plant'].str.startswith(prefix) &
                (group['_merge'] == 'both')
            ]
            if not matches.empty:
                # Assign '620' to the first matching row
                selected_index = matches.index[0]
                df.at[selected_index, '620'] = '620'
                break
    return df

merged_df = assign_620(merged_df)
merged_df.drop(columns=['_merge'], inplace=True)
updated_bps_df = merged_df[merged_df['AMOUNT - UNIT COST'] != 0].groupby(['Item', 'SUPPLIER NUMBER'])['Branch Plant'].agg(lambda x: ', '.join(x.dropna().unique())).reset_index()
updated_bps_df.rename(columns={'Branch Plant': 'Updated on BPs'}, inplace=True)

In [9]:
# Get 'Old Price' and 'Old Price Effective Date'
old_price_info = merged_df.dropna(subset=['DATE - EFFECTIVE']).sort_values('DATE - EFFECTIVE').groupby(['Item', 'SUPPLIER NUMBER']).agg({
    'AMOUNT - UNIT COST': 'last',
    'DATE - EFFECTIVE': 'last'
}).reset_index()
old_price_info.rename(columns={
    'AMOUNT - UNIT COST': 'Old Price',
    'DATE - EFFECTIVE': 'Old Price Effective Date'
}, inplace=True)

In [ ]:
# Aggregate
grouped_df = merged_df.groupby(['Item', 'SUPPLIER NUMBER'], as_index=False).agg({
    'New Price': 'first',
    'CURRENCY CODE - FROM': 'first',
    'SUPPLIER DESCRIPTION': 'first',
    'Branch Plant': lambda x: ', '.join(x.dropna().unique()),
    'UNIT OF MEASURE AS INPUT': 'first',
    'Effective date': 'first',
    'Main Reason 1': 'first',
    'Weight (%) Reason 1': 'first',
    'Main Reason 2': 'first',
    'Weight (%) Reason 2': 'first',
    'Main Reason 3': 'first',
    'Weight (%) Reason 3': 'first',
    '620': 'first'
})

In [11]:
# Merge back 'Old Price', 'Old Price Effective Date', and 'Updated on BPs'
grouped_df = grouped_df.merge(old_price_info, on=['Item', 'SUPPLIER NUMBER'], how='left')
grouped_df = grouped_df.merge(updated_bps_df, on=['Item', 'SUPPLIER NUMBER'], how='left')

In [12]:
# Rename columns
grouped_df.rename(columns={
    'Item': 'Item Number',
    'SUPPLIER DESCRIPTION': 'Supplier Description',
    'SUPPLIER NUMBER': 'Supplier Number',
    'CURRENCY CODE - FROM': 'Currency',
    'UNIT OF MEASURE AS INPUT': 'UoM',
    'Effective date': 'Effective Date'
}, inplace=True)

In [13]:
# Format dates
grouped_df['Effective Date'] = pd.to_datetime(grouped_df['Effective Date']).dt.strftime('%d.%m.%Y')
grouped_df['Old Price Effective Date'] = pd.to_datetime(grouped_df['Old Price Effective Date']).dt.strftime('%d.%m.%Y')

In [ ]:
# Calculate '% Change'
grouped_df['Old Price'] = grouped_df['Old Price'].astype(float).fillna(0.0)
grouped_df['% Change'] = grouped_df.apply(
    lambda row: ((row['New Price'] - row['Old Price']) / row['Old Price'] * 100) if row['Old Price'] != 0 else 0.0,
    axis=1
)
grouped_df['% Change'] = grouped_df['% Change'].round(2)

In [ ]:
# Replace infinities and NaNs
grouped_df.replace([float('inf'), float('-inf'), pd.NA, None], '', inplace=True)

In [ ]:
# Set the order
ordered_columns = ['Effective Date', 'Supplier Description', 'Supplier Number', 'Item Number', 'Branch Plant',
                   'Updated on BPs', 'New Price', 'Old Price', '% Change', 'UoM', 'Currency',
                   'Main Reason 1', 'Weight (%) Reason 1', 'Main Reason 2', 'Weight (%) Reason 2',
                   'Main Reason 3', 'Weight (%) Reason 3', 'Old Price Effective Date', '620']
final_df = grouped_df[ordered_columns]
for col in ['Supplier Number', 'Branch Plant', 'Updated on BPs', 'Item Number', '620']:
    final_df[col] = final_df[col].astype(str)

In [18]:
# Save to Excel
current_datetime = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
output_filename = f'Updated_NewData_{current_datetime}.xlsx'
output_path = os.path.join(output_folder_path, output_filename)
sheet_name = datetime.now().strftime('%d-%m-%Y')
with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
    final_df.to_excel(writer, sheet_name=sheet_name, index=False)
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]

In [ ]:
# Define formats
header_format = workbook.add_format({'bold': True, 'align': 'center', 'border': 1})
old_price_header_format = workbook.add_format({'bold': True, 'align': 'center', 'bg_color': '#FFCCCC', 'border': 1})
new_price_header_format = workbook.add_format({'bold': True, 'align': 'center', 'bg_color': '#CCFFCC', 'border': 1})
grey_header_format = workbook.add_format({'bold': True, 'align': 'center', 'bg_color': '#D3D3D3', 'border': 1})
percentage_format = workbook.add_format({'num_format': '0.00', 'align': 'center', 'border': 1})
price_format = workbook.add_format({'num_format': '0.0000', 'align': 'center', 'border': 1})
cell_format = workbook.add_format({'align': 'center', 'border': 1})

# Write headers
for col_num, value in enumerate(ordered_columns):
    if value == 'Old Price':
        worksheet.write(0, col_num, value, old_price_header_format)
    elif value == 'New Price':
        worksheet.write(0, col_num, value, new_price_header_format)
    elif value in ['Updated on BPs', 'Old Price Effective Date', '% Change']:
        worksheet.write(0, col_num, value, grey_header_format)
    else:
        worksheet.write(0, col_num, value, header_format)

# Set column widths and formats
for i, col in enumerate(ordered_columns):
    max_len = max(final_df[col].astype(str).map(len).max(), len(col)) + 2
    if col in ['New Price', 'Old Price']:
        worksheet.set_column(i, i, max_len, price_format)
    elif col == '% Change':
        worksheet.set_column(i, i, max_len, percentage_format)
    else:
        worksheet.set_column(i, i, max_len, cell_format)

worksheet.autofilter(0, 0, len(final_df), len(ordered_columns) - 1)

print(f"Data has been merged and saved to '{output_path}'")

Data has been merged and saved to 'C:\Users\plmala\OneDrive - Coloplast A S\Desktop\Python\PPM\Price Updates (Python)\0_Price Updates\New Files\Updated_NewData_2024-11-02_20-01-23.xlsx'


In [35]:
# Create Manual Update File
manual_update_columns = ['Supplier Name', 'Supplier Number', 'Item Number', 'Item Description', 'New price',
                         'Item Price Currency', 'Branch Plant', 'Unit of measure', 'Main Reason 1', 'Weight (%) Reason 1',
                         'Main Reason 2', 'Weight (%) Reason 2', 'Main Reason 3', 'Weight (%) Reason 3',
                         'Out-going Item number', 'Quantity Breaks', 'Project name', 'CM05 BP Currency',
                         'Effective date (DDMMYYYY)', 'Update date', 'Item Responsible', 'CM05', 'CM05 BP',
                         'Requester', 'Supp No']
manual_update_df = pd.DataFrame(columns=manual_update_columns)
manual_update_df['Supplier Name'] = final_df['Supplier Description']
manual_update_df['Supplier Number'] = final_df['Supplier Number']
manual_update_df['Item Number'] = final_df['Item Number']
manual_update_df['Item Description'] = ''
manual_update_df['New price'] = final_df['New Price']
manual_update_df['Item Price Currency'] = final_df['Currency']
manual_update_df['Branch Plant'] = final_df['Branch Plant']
manual_update_df['Unit of measure'] = final_df['UoM']
manual_update_df['Main Reason 1'] = final_df['Main Reason 1']
manual_update_df['Weight (%) Reason 1'] = final_df['Weight (%) Reason 1']
manual_update_df['Main Reason 2'] = final_df['Main Reason 2']
manual_update_df['Weight (%) Reason 2'] = final_df['Weight (%) Reason 2']
manual_update_df['Main Reason 3'] = final_df['Main Reason 3']
manual_update_df['Weight (%) Reason 3'] = final_df['Weight (%) Reason 3']
manual_update_df['Out-going Item number'] = ''
manual_update_df['Quantity Breaks'] = ''
manual_update_df['Project name'] = ''
manual_update_df['CM05 BP Currency'] = ''
manual_update_df['Effective date (DDMMYYYY)'] = final_df['Effective Date']
manual_update_df['Update date'] = datetime.now().strftime('%d.%m.%Y')
manual_update_df['Item Responsible'] = ''
manual_update_df['CM05'] = ''
manual_update_df['CM05 BP'] = ''
manual_update_df['Requester'] = ''
manual_update_df['Supp No'] = final_df['620']
manual_update_df.replace(0, '', inplace=True)
manual_update_df.replace('0', '', inplace=True)
manual_update_output_filename = f'Manual_Update_{current_datetime}.xlsx'
manual_update_output_path = os.path.join(manual_update_folder_path, manual_update_output_filename)

with pd.ExcelWriter(manual_update_output_path, engine='xlsxwriter') as writer:
    manual_update_df.to_excel(writer, index=False)
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    header_format = workbook.add_format({'bold': True, 'align': 'center', 'border': 1})
    price_format = workbook.add_format({'num_format': '0.0000', 'align': 'center'})
    date_format = workbook.add_format({'num_format': 'dd.mm.yyyy', 'align': 'center'})
    cell_format = workbook.add_format({'align': 'center'})
    for col_num, value in enumerate(manual_update_df.columns.values):
        worksheet.write(0, col_num, value, header_format)
    for col_num, col_name in enumerate(manual_update_df.columns.values):
        max_len = max(manual_update_df[col_name].astype(str).map(len).max(), len(col_name)) + 2
        if col_name == 'New price':
            worksheet.set_column(col_num, col_num, max_len, price_format)
        elif col_name in ['Effective date (DDMMYYYY)', 'Update date']:
            worksheet.set_column(col_num, col_num, max_len, date_format)
        else:
            worksheet.set_column(col_num, col_num, max_len, cell_format)

    worksheet.autofilter(0, 0, manual_update_df.shape[0], manual_update_df.shape[1] - 1)

print(f"Manual update file has been created at '{manual_update_output_path}'")

Manual update file has been created at 'C:\Users\plmala\OneDrive - Coloplast A S\Desktop\Python\PPM\Price Updates (Python)\0_Price Updates\Manual Update\Manual_Update_2024-11-02_20-01-23.xlsx'


In [34]:
# Create API Template File
api_template_columns = ['Effective date (DDMMYYYY)', 'Supplier Name', 'Supplier Number', 'Item Number',
                        'Quantity Breaks', 'New price', 'Item Price Currency', 'Unit of measure', 'Branch Plant',
                        'Main Reason 1', 'Weight (%) Reason 1', 'Main Reason 2', 'Weight (%) Reason 2',
                        'Main Reason 3', 'Weight (%) Reason 3', 'Out-going Item number', 'Project name',
                        'Comments / Notes']
api_template_df = pd.DataFrame(columns=api_template_columns)
api_template_df['Effective date (DDMMYYYY)'] = final_df['Effective Date']
api_template_df['Supplier Name'] = final_df['Supplier Description']
api_template_df['Supplier Number'] = final_df['Supplier Number']
api_template_df['Item Number'] = final_df['Item Number']
api_template_df['Quantity Breaks'] = ''
api_template_df['New price'] = final_df['New Price']
api_template_df['Item Price Currency'] = final_df['Currency']
api_template_df['Unit of measure'] = final_df['UoM']
api_template_df['Branch Plant'] = final_df['Branch Plant']
api_template_df['Main Reason 1'] = final_df['Main Reason 1']
api_template_df['Weight (%) Reason 1'] = final_df['Weight (%) Reason 1']
api_template_df['Main Reason 2'] = final_df['Main Reason 2']
api_template_df['Weight (%) Reason 2'] = final_df['Weight (%) Reason 2']
api_template_df['Main Reason 3'] = final_df['Main Reason 3']
api_template_df['Weight (%) Reason 3'] = final_df['Weight (%) Reason 3']
api_template_df['Out-going Item number'] = ''
api_template_df['Project name'] = ''
api_template_df['Comments / Notes'] = ''
api_template_df.replace(0, '', inplace=True)
api_template_df.replace('0', '', inplace=True)
api_template_output_filename = f'API_Template_{current_datetime}.xlsx'
api_template_output_path = os.path.join(api_template_folder_path, api_template_output_filename)

with pd.ExcelWriter(api_template_output_path, engine='xlsxwriter') as writer:
    api_template_df.to_excel(writer, index=False)
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    header_format = workbook.add_format({'bold': True, 'align': 'center', 'border': 1})
    price_format = workbook.add_format({'num_format': '0.0000', 'align': 'center'})
    date_format = workbook.add_format({'num_format': 'dd.mm.yyyy', 'align': 'center'})
    cell_format = workbook.add_format({'align': 'center'})

    for col_num, value in enumerate(api_template_df.columns.values):
        worksheet.write(0, col_num, value, header_format)
    for col_num, col_name in enumerate(api_template_df.columns.values):
        max_len = max(api_template_df[col_name].astype(str).map(len).max(), len(col_name)) + 2
        if col_name == 'New price':
            worksheet.set_column(col_num, col_num, max_len, price_format)
        elif col_name == 'Effective date (DDMMYYYY)':
            worksheet.set_column(col_num, col_num, max_len, date_format)
        else:
            worksheet.set_column(col_num, col_num, max_len, cell_format)

    worksheet.autofilter(0, 0, api_template_df.shape[0], api_template_df.shape[1] - 1)

print(f"API template file has been created at '{api_template_output_path}'")

API template file has been created at 'C:\Users\plmala\OneDrive - Coloplast A S\Desktop\Python\PPM\Price Updates (Python)\0_Price Updates\API Template\API_Template_2024-11-02_20-01-23.xlsx'


In [ ]:
# Create Massive Upload File
massive_upload_columns = ['Supplier Number', '2nd Item Number', 'Unit Cost', 'Cur Cod',
                          'Business Unit', 'UM', 'Effective Date', 'Expired Date',
                          'Quantity Ordered', 'Supp No']
massive_upload_df = pd.DataFrame(columns=massive_upload_columns)
massive_upload_df['Supplier Number'] = final_df['Supplier Number']
massive_upload_df['2nd Item Number'] = final_df['Item Number']
massive_upload_df['Unit Cost'] = final_df['New Price']
massive_upload_df['Cur Cod'] = final_df['Currency']
massive_upload_df['Business Unit'] = final_df['Branch Plant']
massive_upload_df['UM'] = final_df['UoM']
massive_upload_df['Effective Date'] = final_df['Effective Date']
massive_upload_df['Expired Date'] = '31.12.2040'
massive_upload_df['Quantity Ordered'] = ''
massive_upload_df['Supp No'] = final_df['620']
massive_upload_df.replace(0, '', inplace=True)
massive_upload_df.replace('0', '', inplace=True)
massive_upload_output_filename = f'Massive_Upload_{current_datetime}.xlsx'
massive_upload_output_path = os.path.join(massive_upload_folder_path, massive_upload_output_filename)

with pd.ExcelWriter(massive_upload_output_path, engine='xlsxwriter') as writer:
    massive_upload_df.to_excel(writer, index=False)
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    header_format = workbook.add_format({'bold': True, 'align': 'center', 'border': 1})
    price_format = workbook.add_format({'num_format': '0.0000', 'align': 'center'})
    date_format = workbook.add_format({'num_format': 'dd.mm.yyyy', 'align': 'center'})
    cell_format = workbook.add_format({'align': 'center'})

    for col_num, value in enumerate(massive_upload_df.columns.values):
        worksheet.write(0, col_num, value, header_format)
    for col_num, col_name in enumerate(massive_upload_df.columns.values):
        max_len = max(massive_upload_df[col_name].astype(str).map(len).max(), len(col_name)) + 2
        if col_name == 'Unit Cost':
            worksheet.set_column(col_num, col_num, max_len, price_format)
        elif col_name in ['Effective Date', 'Expired Date']:
            worksheet.set_column(col_num, col_num, max_len, date_format)
        else:
            worksheet.set_column(col_num, col_num, max_len, cell_format)

    worksheet.autofilter(0, 0, massive_upload_df.shape[0], massive_upload_df.shape[1] - 1)

print(f"Massive upload file has been created at '{massive_upload_output_path}'")

Massive upload file has been created at 'C:\Users\plmala\OneDrive - Coloplast A S\Desktop\Python\PPM\Price Updates (Python)\0_Price Updates\Massive Upload\Massive_Upload_2024-11-02_20-01-23.xlsx'
